# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    14-Jan-2026**

**News Source / Catalyst:** Yen tumbles and Japan stocks soar as election talk reignites ‘Takaichi trade’

**Markets (s) Impacted:** FX, EQ, FI

**Trade Taxonomy:** RV Macro / Structural FX / Event-Driven

**Trade Idea (1 sentence):**
> Exexute a Long Nikkei (USD-hedged) vs. Short JGB (10Y) position to capitalise of the 'Takaichi Trade' - capturing equity upside from a JPY 21.3 tn stimulus package while shielding against a depreciating Yen and hedging the record rise in sovereign risk premiums. 

**Client/Desk Pitch (30 seconds):**
> The Japanese market is currently driven by a violend decoupling of fiscal and monetary policy. With the Nikkei at record highs and 10y yields hitting a 27-year peak of 2.16%, the 23-Jan-2026 snap election serves as the ultimate catalyst for Takaichi's proposed JPY 21.4 tn stimulus. While this is rocketing Tech and Defence, it has pushed the Yen to the JPY 160 intervention zone and crushing bond prices. We are recommending a Long DXJ (Hedged Nikkei) / Short 10Y JGB relative value spread capturing the agressive fiscal-driven equity upside while stripping out Yen devaluation risk and profiting directly from the rising sovereign risk premium. It's a pure-play on Japan's historic fiscal expansion that isolates the alpha from the currency volatility.

**Why Now:**
- Election Catalyst: Markets are front-running the January 23rd parliament dissolution to lock in the "Takaichi Trade" before the February vote
- Technical Breakout: The Nikkei has breached the 54,000 psychological barrier while 10Y yields hit 27-year highs-momentum is now non-linear
- Intervention Edge: With the Yen at 159, we are at the limit of verbal warnings; entering now captures the final melt-up before a central bank reset


---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**
- Fiscal Monetary Divergence: Massive JPY 21tn stimulus (fiscal) is clashing with a dovish tethered Bank of Japan (monetary), fueling a record -breaking equity rally
- Sovereign Risk Premium: Expanding debt-to-GDP (200%+) is finally forcing a "debt deluge" in the bond market pushing JGB yields to 27-year highs
- Intervention Threshold: The Yen is testing a "line in the sand" at JPY 160, creating a high conviction environment for hedged equities over unhedged exposure

**Long Leg (Tickers + Rationale):**
- DXJ (WisdomTree Japan Hedged Equity)

**Short Leg (Tickers + Rationale):**
- XJSE.DE (liquid JGB Proxy)

**Weights:** Beta Neutral (Volatility Weighted)


---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |        S          |       |
| Oil Beta             |        W          |       |
| Rates Duration       |        S          |       |
| Growth vs Value      |        M          |       |
| Risk-On/Off          |        S          |       |
| China/EM Beta        |        W          |       |
| Volatility Regime    |        S          |       |

**Catalyst Type:** Regime Shift

**Realisation Window:** Weeks (heading into 23-Jan-2026)

**Holding Period Selected:**  45 days

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:**
- Long leg (DXJ): 250 units per $100k (weighted at ~38% notional to account for higher equity volatility)
- Short leg (JGB Proxy): 1,050 units per. $100k (weighted at ~62% notional to balance the lower volatility on the macro leg)

**Stop-Loss:** -2.5% 

**Take-Profit (optional):**  6%

**Re-entry:** Yes

**FX-Hedging Required:** Yes

**Liquidity Notes:** Spreads are tightest during 13:00-16:00 GMT overlap (London/NY). Avoid trading during the Tokyo-only session to minimise slippage on the USD-hedged leg.  

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information (Primary: Takaichi delays the snap election or the BoJ surpises with a hawkish 50bps hike)

---


# **PYTHON IMPLEMENTATION**

In [36]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [47]:
# --- Parameters ---

long_tickers = ["DXJ"]    
short_tickers = ["XJSE.DE"]                    

start_date = "2020-01-01"                         # Backtest window
holding_period = 45                               # days
stop_loss = -0.025                                 # -2% stop
take_profit = 0.06                                # optional e.g. +0.03 for +3%
long_positioning = 0.38
short_positioning = 0.62

# --- Weightings ---

weights_long = long_positioning / (long_positioning + short_positioning)
weights_short = short_positioning / (long_positioning + short_positioning)

In [48]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()


/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_6596/911793947.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  2 of 2 completed


In [49]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [50]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
29,2025-01-22,-0.0378
30,2025-03-26,-0.0342
31,2025-05-29,0.0645
32,2025-07-31,0.0614
33,2025-10-02,0.0751


In [51]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         34.0000
Win Rate              0.4706
Average PnL           0.0070
Best Trade            0.0890
Worst Trade          -0.1347
Stop-Out Frequency    0.5294
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 34.0000
- Win Rate:     0.4706
- Avg PnL:      0.0070
- Best:         0.0890
- Worst:        -0.1347
- Stop-Outs:    0.5294

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [52]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [53]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [54]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
29,RV-4A7C9310,2026-01-14 11:41:27,Macro RV Spread,RV Macro Spread,DXJ,XJSE.DE,2025-01-22,2025-02-07,36.2955,35.6158,-3.7847,-2.5000,6.0000,STOPPED,16,
30,RV-9B859E18,2026-01-14 11:41:27,Macro RV Spread,RV Macro Spread,DXJ,XJSE.DE,2025-03-26,2025-03-28,38.1244,36.9106,-3.4178,-2.5000,6.0000,STOPPED,2,
31,RV-000C1C25,2026-01-14 11:41:27,Macro RV Spread,RV Macro Spread,DXJ,XJSE.DE,2025-05-29,2025-07-30,37.8940,40.7422,11.8257,-2.5000,6.0000,TAKE_PROFIT,62,
32,RV-2EA24F33,2026-01-14 11:41:27,Macro RV Spread,RV Macro Spread,DXJ,XJSE.DE,2025-07-31,2025-10-01,40.9669,43.9130,8.2774,-2.5000,6.0000,TAKE_PROFIT,62,
33,RV-8D69768A,2026-01-14 11:41:27,Macro RV Spread,RV Macro Spread,DXJ,XJSE.DE,2025-10-02,2025-12-03,44.0025,48.7678,16.3576,-2.5000,6.0000,TAKE_PROFIT,62,


In [55]:
blotter.to_csv("trade_blotter.csv", index=False)